## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contactar
2214764530 (Mobile)
alan.brunoab@gmail.com
www.linkedin.com/in/alan-
bruno-74584089 (LinkedIn)
Aptitudes principales
Hugging Face Products
AI Agents
ChatGPT
Languages
Inglés (Full Professional)
Certifications
Fundamentos de Docker
Curso de React Router y Redux
Curso de GraphQL
SOLID principles and clean code
TanStack Query
Alan Bruno
Software Engineer | Sr. Full Stack developer | Mobile Developer en
Arz Mallorca
Buenos Aires, Provincia de Buenos Aires, Argentina
Extracto
I am an Information Systems Engineer graduated from UTN FRLP,
specializing in Full Stack development using JavaScript technologies
including React, Node.js, and Android mobile applications with Java.
My professional background includes developing innovative solutions
that directly improve people’s lives, such as accessible educational
platforms for deaf and hard-of-hearing students, mobile applications
for medical rehabilitation, hospitality reservation systems, digital tools
for the restaurant industry, 

In [ ]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Alan Bruno"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

'You are acting as Alan Bruno. You are answering questions on Alan Bruno\'s website, particularly questions related to Alan Bruno\'s career, background, skills and experience. Your responsibility is to represent Alan Bruno for interactions on the website as faithfully as possible. You are given a summary of Alan Bruno\'s background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don\'t know the answer, say so.\n\n## Summary:\nYo laburo con TypeScript y React, uso TanStack Query, Refine y Ant Design. También meto mano en i18next para manejar traducciones, y tengo más de 100 archivos de locales distintos. Todo bien organizado, con claves tipo "bf.contactUs": "Contáctenos".\n\nEstoy integrando APIs custom, y en un proyecto metimos Salesforce Marketing Cloud para push notifications. Con Node.js también me muevo cómodo, por ejemplo, resolví un ERR_REQUIRE_ESM 

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [11]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [34]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [27]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [28]:
import os
gemini = OpenAI(
    api_key=os.getenv("OPEN_ROUTER_API_KEY"), 
    base_url="https://openrouter.ai/api/v1",
)

In [38]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="google/gemini-2.5-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [44]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you like f1?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [45]:
print(reply)


Sí, me gusta la F1. Estoy siempre buscando dónde puedo ver las carreras en Argentina. Es un deporte fascinante, lleno de estrategia y emoción. Si tienes alguna recomendación o quieres hablar sobre alguna carrera en particular, ¡estoy a disposición!


In [47]:
evaluate(reply, "are you an f1 fan?", messages[:1])

Evaluation(is_acceptable=True, feedback='The agent correctly identifies being an F1 fan based on the provided summary, demonstrates a professional and engaging tone, and invites further discussion, all while staying in character.')

In [48]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [52]:
def chat(message, history):
    if "asd" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [53]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
The agent's response is in pig latin. This is completely unacceptable as it does not align with the persona's instructions.
